In [9]:
from ollama import chat
from ollama import ChatResponse


def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a (int): The first number
    b (int): The second number

  Returns:
    int: The sum of the two numbers
  """
  return a + b


def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  return a - b


# Tools can still be manually defined and passed into chat
subtract_two_numbers_tool = {
  'type': 'function',
  'function': {
    'name': 'subtract_two_numbers',
    'description': 'Subtract two numbers',
    'parameters': {
      'type': 'object',
      'required': ['a', 'b'],
      'properties': {
        'a': {'type': 'integer', 'description': 'The first number'},
        'b': {'type': 'integer', 'description': 'The second number'},
      },
    },
  },
}

messages = [{'role': 'user', 'content': 'What are transformers in AI?'}] #What is three plus one?
print('Prompt:', messages[0]['content'])

available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}

response: ChatResponse = chat(
  'llama3.2',
  messages=messages,
  tools=[add_two_numbers, subtract_two_numbers_tool],
)

print(response)

if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_functions.get(tool.function.name):
      print('Calling function:', tool.function.name)
      print('Arguments:', tool.function.arguments)
      output = function_to_call(**tool.function.arguments)
      print('Function output:', output)
    else:
      print('Function', tool.function.name, 'not found')


      
# Only needed to chat with the model using the tool call results
if response.message.tool_calls:
  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  final_response = chat('llama3.2', messages=messages)
  print('Final response:', final_response.message.content)

else:
  print('No tool calls returned from model')

Prompt: What are transformers in AI?
model='llama3.2' created_at='2024-12-20T22:34:21.409618Z' done=True done_reason='stop' total_duration=1036800583 load_duration=43330333 prompt_eval_count=233 prompt_eval_duration=713000000 eval_count=24 eval_duration=278000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='add_two_numbers', arguments={'a': 0, 'b': 10}))])
Final response: Transformers are a type of neural network architecture that have revolutionized the field of natural language processing (NLP) and machine learning. They were introduced in 2017 by Vaswani et al. and have since become one of the most popular architectures for NLP tasks.

The core idea behind transformers is to replace traditional recurrent neural networks (RNNs) with self-attention mechanisms, which allow the model to weigh the importance of different input elements relative to each other. This allows the model to capture long-range dependencies in the data, r

In [7]:
response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

print(response) 
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

model='llama3.2' created_at='2024-12-20T22:31:06.57676Z' done=True done_reason='stop' total_duration=4424299750 load_duration=30618000 prompt_eval_count=31 prompt_eval_duration=661000000 eval_count=312 eval_duration=3731000000 message=Message(role='assistant', content="The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. Sunlight enters Earth's atmosphere and consists of a spectrum of colors, including red, orange, yellow, green, blue, indigo, and violet.\n2. When sunlight interacts with tiny molecules of gases such as nitrogen (N2) and oxygen (O2) in the air, it scatters in all directions.\n3. The shorter (blue) wavelengths of light are scattered more than the longer (red) wavelengths because they encounter more collisions with the smaller molecules. This is known as Rayleigh scattering.\n4. As a result, the blue light is dispersed through

In [28]:
from ollama import chat
from pydantic import BaseModel


# Define the schema for the response
class FriendInfo(BaseModel):
  name: str
  age: int
  is_available: bool


class FriendList(BaseModel):
  friends: list[FriendInfo]


# schema = {'type': 'object', 'properties': {'friends': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'is_available': {'type': 'boolean'}}, 'required': ['name', 'age', 'is_available']}}}, 'required': ['friends']}
response = chat(
  model='llama3.2',
  messages=[{'role': 'user', 'content': 'I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format'}],
  format=FriendList.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
  options={'temperature': 0},  # Make responses more deterministic
)

# Use Pydantic to validate the response
friends_response = FriendList.model_validate_json(response.message.content)
print(friends_response)

friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)]


In [23]:

from ollama import embed

response = embed(model='llama3.2', input='React AI Optimizer')
print(response['embeddings'])
diff_embedd = embed(model='llama3.2', input=diff2)
print(diff_embedd['embeddings'])


[[-0.028120965, 0.00452322, 0.016376976, -0.008837591, 0.0034958783, 0.014925992, 0.023899702, -0.01289475, -0.0075584995, -0.017641868, 0.01239418, -0.0047272933, 0.0014071208, 0.0379007, -0.018868439, 0.014229236, -0.003727954, 0.012907992, 0.004204622, 0.005020302, -0.01360067, -0.018105455, -0.018668074, -0.01186196, 0.0035334397, -0.00418778, 0.014299679, -0.023899032, -0.0049061114, -0.007628676, 0.022433117, 0.013888519, 0.0028002777, -0.00683959, 0.013760059, -0.028211145, 0.016522497, 0.011339007, -0.018877532, 0.005561468, 0.0075507695, -0.00015916182, 0.021492993, 0.01786265, -0.0009128877, -0.0013726398, -0.008886366, 0.008887761, -0.027842483, 0.0026407274, 0.03312036, 0.023960572, 8.6318265e-05, 0.0023572173, 0.002196492, 0.0021481356, -0.009234923, -0.018791495, 0.0006025226, 0.023042452, 0.011947352, -0.0030084734, -0.008580277, -0.0055007157, 0.030913873, -0.03460156, 0.010271063, 0.009660783, -0.022077966, -0.03726212, 0.006150695, 0.003998229, 0.014201436, -0.0135568

In [26]:
import numpy as np
from typing import List

def cosine_similarity(a: List[float], b: List[float]) -> float:
    """Calculate cosine similarity between two vectors"""
    # Convert to numpy arrays
    a_vec = np.array(a)
    b_vec = np.array(b)
    
    # Calculate dot product and magnitudes
    dot_product = np.dot(a_vec, b_vec)
    #a_norm = np.linalg.norm(a_vec) 
    #b_norm = np.linalg.norm(b_vec)
    
    # Return cosine similarity
    #return dot_product / (a_norm * b_norm)
    return dot_product
# Calculate similarity between first vector and itself (should be 1.0)
similarity = cosine_similarity(response['embeddings'][0], diff_embedd['embeddings'][0])
print(f"Cosine similarity: {similarity}")


Cosine similarity: 0.07287793454265126


In [17]:
diff2 = """
    diff --git a/git-ai-commit.sh b/git-ai-commit.sh
new file mode 100755
index 0000000000..d7cf9c8873
--- /dev/null
+++ b/git-ai-commit.sh
@@ -0,0 +1,57 @@
+#!/bin/bash
+
+# Check if files are passed as arguments or if the user wants to add all changes
+if [ "$#" -eq 0 ]; then
+  echo "No files specified. Staging all changes..."
+  git add .
+else
+  echo "Staging specified files..."
+  git add "$@"
+fi
+
+# Check if there are any staged changes
+diff=$(git diff --cached | cat)
+if [ -z "$diff" ]; then
+  echo "No staged changes to commit."
+  exit 1
+fi
+
+# Escape the diff for JSON compatibility
+escaped_diff=$(echo "$diff" | jq -Rsa .)
+
+# Prepare the payload for Ollama
+data=$(jq -n --arg diff "$escaped_diff" '{
+  "model": "llama3.2",
+  "stream": false,
+  "prompt": ("" + $diff)
+}')
+
+# Call the Ollama API to generate the commit message
+response=$(curl -s -X POST http://localhost:11434/api/generate -d "$data" -H "Content-Type: application/json")
+
+# Extract the commit message from the API response
+commit_message=$(echo "$response" | jq -r '.response')
+
+# Check if a commit message was generated
+if [ -z "$commit_message" ]; then
+  echo "Error: Failed to generate a commit message."
+  exit 1
+fi
+
+# Display the generated commit message in the console
+echo "AI-Generated Commit Message:"
+echo "--------------------------------------"
+echo "$commit_message"
+echo "--------------------------------------"
+
+# Confirm with the user before committing
+read -p "Do you want to use this commit message? (y/n): " confirm
+if [[ "$confirm" != "y" ]]; then
+  echo "Commit aborted."
+  exit 1
+fi
+
+# Use the generated commit message directly in the commit
+git commit -m "$commit_message"
+
+echo "Commit created successfully with AI-generated message!"
\ No newline at end of file
diff --git a/packages/react/docs/ai-optimization.md b/packages/react/docs/ai-optimization.md
new file mode 100644
index 0000000000..fc725e8385
--- /dev/null
+++ b/packages/react/docs/ai-optimization.md
@@ -0,0 +1,16 @@
+
+# AI-Powered Code Optimization
+
+This module provides real-time code analysis and optimization suggestions using AI.
+
+## Features
+- Real-time code analysis
+- Performance metrics collection
+- AI-powered suggestions
+- Pattern detection
+
+## Architecture
+- AICodeOptimizer: Core orchestration
+- CodeAnalyzer: Static analysis
+- MetricsCollector: Runtime metrics
+
diff --git a/packages/react/src/ai/optimizer.ts b/packages/react/src/ai/optimizer.ts
new file mode 100644
index 0000000000..0e1fcd4606
--- /dev/null
+++ b/packages/react/src/ai/optimizer.ts
@@ -0,0 +1,18 @@
+
+import { OpenAI } from "./ai/openai";
+import { CodeAnalyzer } from "./analysis/code";
+import { MetricsCollector } from "./metrics/collector";
+
+export class AICodeOptimizer {
+  constructor() {
+    this.ai = new OpenAI();
+    this.analyzer = new CodeAnalyzer();
+    this.metrics = new MetricsCollector();
+  }
+
+  async analyze(code: string) {
+    const metrics = await this.metrics.collect(code);
+    const analysis = await this.analyzer.analyze(code);
+    return this.ai.suggest(metrics, analysis);
+  }
+}
diff --git a/packages/react/src/analysis/code.ts b/packages/react/src/analysis/code.ts
new file mode 100644
index 0000000000..66ae5494fd
--- /dev/null
+++ b/packages/react/src/analysis/code.ts
@@ -0,0 +1,10 @@
+
+export class CodeAnalyzer {
+  analyze(code: string) {
+    return {
+      complexity: this.calculateComplexity(code),
+      patterns: this.detectPatterns(code),
+      issues: this.findIssues(code)
+    };
+  }
+}
diff --git a/packages/react/src/components/CodeOptimizationPanel.tsx b/packages/react/src/components/CodeOptimizationPanel.tsx
new file mode 100644
index 0000000000..cacb561952
--- /dev/null
+++ b/packages/react/src/components/CodeOptimizationPanel.tsx
@@ -0,0 +1,22 @@
+
+import { AICodeOptimizer } from "../ai/optimizer";
+import { useEffect, useState } from "react";
+
+export const CodeOptimizationPanel = () => {
+  const [optimizer] = useState(() => new AICodeOptimizer());
+  const [suggestions, setSuggestions] = useState([]);
+
+  useEffect(() => {
+    optimizer.analyze(currentCode)
+      .then(setSuggestions);
+  }, [currentCode]);
+
+  return (
+    <div className="optimization-panel">
+      <h2>AI Suggestions</h2>
+      {suggestions.map(suggestion => (
+        <SuggestionCard key={suggestion.id} {...suggestion} />
+      ))}
+    </div>
+  );
+}
diff --git a/packages/react/src/config/ai.ts b/packages/react/src/config/ai.ts
new file mode 100644
index 0000000000..d0cc0c275c
--- /dev/null
+++ b/packages/react/src/config/ai.ts
@@ -0,0 +1,14 @@
+
+export const config = {
+  ai: {
+    model: "gpt-4",
+    temperature: 0.2,
+    maxTokens: 1000
+  },
+  analysis: {
+    thresholds: {
+      complexity: 20,
+      issues: 5
+    }
+  }
+}
diff --git a/packages/react/src/metrics/collector.ts b/packages/react/src/metrics/collector.ts
new file mode 100644
index 0000000000..1f544f7352
--- /dev/null
+++ b/packages/react/src/metrics/collector.ts
@@ -0,0 +1,10 @@
+
+export class MetricsCollector {
+  collect(code: string) {
+    return {
+      performance: this.measurePerformance(code),
+      memory: this.analyzeMemory(code),
+      dependencies: this.checkDependencies(code)
+    };
+  }
+}
diff --git a/packages/react/src/types/analysis.ts b/packages/react/src/types/analysis.ts
new file mode 100644
index 0000000000..c52d0a4f85
--- /dev/null
+++ b/packages/react/src/types/analysis.ts
@@ -0,0 +1,15 @@
+
+export interface CodeAnalysis {
+  complexity: {
+    cognitive: number;
+    cyclomatic: number;
+  };
+  patterns: {
+    antiPatterns: string[];
+    improvements: string[];
+  };
+  issues: {
+    performance: string[];
+    security: string[];
+  };
+}
   
"""